# PSD Comparison Notebook
This notebook visualizes reliable pore size distributions from precomputed PSD tables to compare multiple scans. No new PSD calculations or segmentations are performed here.

## Imports
We only need pandas for data handling and matplotlib for plotting so that the figures remain publication-ready and dependency-light.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## File paths and labels
Define the three PSD tables along with human-readable labels, consistent colors, and linestyles so the plots are easy to read and interpret.

In [ ]:
base_dir = Path.cwd()
dataset_configs = [
    {
        
: base_dir / 
 / 
,  # update to real paths
        
: 
,
        
: 
,
        
: 

    },
    {
        
: base_dir / 
 / 
,
        
: 
,
        
: 
,
        
: 

    }
]

## Data loading & filtering
Load each CSV, verify the required columns, filter to the reliable diameter bins (per Vogel et al. 2010), and sort by diameter before plotting.

In [ ]:
required_columns = {
    'Diameter_um',
    'Differential_PSD',
    'Cumulative_Porosity',
    'is_reliable',
}
processed_datasets = []
for config in dataset_configs:
    path = config['path']
    if not path.exists():
        raise FileNotFoundError(f'Required PSD file missing: {path}')
    df = pd.read_csv(path)
    missing = required_columns - set(df.columns)
    if missing:
        raise ValueError(f'Missing expected columns {missing} in {path.name}')
    df = df[df['is_reliable']]
    df = df.sort_values('Diameter_um')
    processed_datasets.append({
        'label': config['label'],
        'color': config['color'],
        'linestyle': config['linestyle'],
        'data': df
    })

## Differential PSD plot
Plot the differential PSD curves on a shared logarithmic diameter axis to directly compare the pore size contributions in each dataset.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for ds in processed_datasets:
    ax.plot(
        ds['data']['Diameter_um'],
        ds['data']['Differential_PSD'],
        label=ds['label'],
        color=ds['color'],
        linestyle=ds['linestyle'],
        linewidth=1.8,
    )
ax.set_xscale('log')
ax.set_xlabel('Diameter (μm)')
ax.set_ylabel('Differential PSD')
ax.set_title('Pore Size Distribution Comparison (Reliable Range)')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.legend()
fig.tight_layout()
plt.show()

## Cumulative porosity plot
Overlay cumulative porosity curves with the same logarithmic diameter axis to see how each sample approaches full porosity within the reliable range.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for ds in processed_datasets:
    ax.plot(
        ds['data']['Diameter_um'],
        ds['data']['Cumulative_Porosity'],
        label=ds['label'],
        color=ds['color'],
        linestyle=ds['linestyle'],
        linewidth=1.8,
    )
ax.set_xscale('log')
ax.set_xlabel('Diameter (μm)')
ax.set_ylabel('Cumulative Porosity')
ax.set_ylim(0, 1)
ax.set_title('Cumulative Porosity (Reliable Diameter Range)')
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.legend()
fig.tight_layout()
plt.show()